# Custom Pipelines in EvalML

EvalML pipelines consist of modular components combining any number of transformers and an estimator. This allows you to create pipelines that fit the needs of your data to achieve the best results. You can create your own pipeline like this:

In [1]:
from evalml.pipelines import PipelineBase
from evalml.pipelines.components import StandardScaler, SimpleImputer
from evalml.pipelines.components.estimators import LogisticRegressionClassifier


# objectives can be either a str or the evalml objective object
objective = 'Precision_Macro'


# pipeline needs to be a subclass of `PipelineBase`
class CustomPipeline(PipelineBase):
    # component_graph and problem_types are required class variables
    
    # components can be passed in as objects or as component name strings
    component_graph = ['Simple Imputer', StandardScaler(), 'Logistic Regression Classifier']
    problem_types = ['binary', 'multiclass']
    

pipeline = CustomPipeline(parameters={}, objective=objective, n_jobs=-1, random_state=0)

In [2]:
from evalml.demos import load_wine

X, y = load_wine()

pipeline.fit(X, y)
pipeline.score(X, y)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].